### For detailed information on using the PatentsView API, see:  
* [PatentsView API information page](https://patentsview.org/apis/api-endpoints/patentsbeta)  
* [PatentsView API Query Language](https://patentsview.org/apis/api-query-language)  
* [Swagger API Documentation and test page](https://search.patentsview.org/swagger-ui/)

In [1]:
import pandas as pd
import json
import requests

An API key is required to view results from the PatentsView API.  
To request an API key, please submit the API key request form here:  
https://patentsview.org/apis/keyrequest

In [2]:
api_key = "my_API_key"

Responses from the Patentsview API are formatted as JSON and always contain the key **"error"** .  
Successful responses will also contain the keys:  
  
**"count"** _(the number of records included in the response)_,  
  
**"total_hits"** _(the total number of records in the PatentsView data matching the parameters of the request)_,  
  
and another key containing the actual record data which is specific to each endpoint.  
The function below converts the endpoint to the expected results key.

In [3]:
special_keys = {
    'ipc' : 'ipcr',
    'wipo': 'wipo'
}

def response_key(endpoint:str) -> str:
    """
    Given the API endpoint contacted, returns the json key for the data returned by the API.
    In most cases, this is the plural of the term at the end of the endpoint URL.
    """
    endpoint = endpoint.rstrip('/')
    leaf = endpoint.split('/')[-1]
    if leaf in special_keys:
        return special_keys[leaf]
    elif leaf.endswith('s'):
        return leaf + 'es'
    else:
        return leaf + 's'

### Identify and set your endpoint of interest 

In [4]:
base_url = 'https://search.patentsview.org'

In [5]:
# get the endpoint that matches your query from the Swagger page
endpoint = 'api/v1/patent' # e.g.

### constructing your query:  

| param | name | object type | default |  
| ---: | :--- | :--- | :--- |  
| q | query | dict | None (required parameter) |  
| f | fields | list | depends on endpoint |  
| o |  options | dict | "size": 100, "after": None (optional) |  
| s | sort | list of dict | primary key of table |  

See more detail at the [PatentsView API Query Language](https://patentsview.org/apis/api-query-language) Page

In [6]:
# any of these except q can be omitted
param_dict = {
    "f" : ["patent_id", "patent_title", "patent_date"],
    "o" : {"size":200},
    "q" : {"patent_id":["D345393","10905426","11172927"]},
    "s" : [{"patent_id":"asc"}], 
}

### GET requests:  
parameters provided as part of the url after the endpoint, separated by "&"

In [7]:
param_string = "&".join([f"{param_name}={json.dumps(param_val)}" for param_name, param_val in param_dict.items()])
# note the json.dumps to ensure strings are surrounded by double quotes instead of single
# the api will not accept single quotes
query_url = f"{base_url}/{endpoint.strip('/')}/?{param_string}"

In [8]:
response = requests.get(query_url, headers={"X-Api-Key": api_key})
print(response)
if response.status_code == 200:
    print(pd.Series(response.json())[:-1])
else: 
    print(pd.Series(response.headers)) 

<Response [200]>
error         False
count             3
total_hits        3
dtype: object


In [9]:
response_unpacked = pd.DataFrame(response.json()[response_key(endpoint)])
display(response_unpacked.head())

,patent_id,patent_title,patent_date
0,10905426,Detachable motor powered surgical instrument,2021-02-02
1,11172927,Staple cartridges for forming staples having d...,2021-11-16
2,D345393,Turtle toy car,1994-03-22


### POST requests:  
parameters provided inside a json request body

In [10]:
query_url = f"{base_url}/{endpoint.strip('/')}"
headers = {
    "X-Api-Key": api_key,
    "accept": "application/json",
    "Content-Type": "application/json"
}

In [11]:
response = requests.post(url=query_url, headers=headers, json=param_dict)
# note that the POST request takes the parameter dictionary directly rather than requiring json as text
print(response)
if response.status_code == 200:
    print(pd.Series(response.json())[:-1])
else: 
    print(pd.Series(response.headers)) 

<Response [200]>
error         False
count             3
total_hits        3
dtype: object


In [12]:
response_unpacked = pd.DataFrame(response.json()[response_key(endpoint)])
display(response_unpacked.head())

,patent_id,patent_title,patent_date
0,10905426,Detachable motor powered surgical instrument,2021-02-02
1,11172927,Staple cartridges for forming staples having d...,2021-11-16
2,D345393,Turtle toy car,1994-03-22


### Single Record GET Requests  
Many of the endpoints also allow providing a single id at the end of the endpoint URL to retrieve default information about that record

In [13]:
record_id = "D345393" # e.g. a patent ID for the patents endpoint

In [14]:
query_url = f"{base_url}/{endpoint.strip('/')}/{record_id}/"
response = requests.get(query_url, headers={"X-Api-Key": api_key})
print(response)
if response.status_code == 200:
    print(pd.Series(response.json())[:-1])
else: 
    print(pd.Series(response.headers)) 

<Response [200]>
error         False
count             1
total_hits        1
dtype: object


In [15]:
display(pd.Series(response.json()[response_key(endpoint)][0]))

patent_id              D345393
patent_title    Turtle toy car
patent_date         1994-03-22
dtype: object